# Integrated Gradients for LLMs

First we'll practice collecting some gradients with nnsight.

In [1]:
from nnsight import LanguageModel

model = LanguageModel("meta-llama/Llama-3.2-3B")
model.to_empty(device="mps")
model.device

/opt/miniconda3/envs/tl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='mps', index=0)

In [ ]:
prompt = "1+1="
layer_idx = -1

with model.trace(prompt):
    # access module inputs/outputs during forward pass
    hs = model.model.layers[layer_idx].output[0].save()
    logits = model.output[0].save()
    
    # enter backward() context
    with logits.sum().backward():
        # You can access gradients within backward context
        grad = logits.grad.save()

print(model.tokenizer.decode(logits.argmax()))
print(grad)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.94s/it]


In [ ]:
from captum.attr import (
    LayerIntegratedGradients,
    LLMAttribution,
    LLMGradientAttribution,
    TextTokenInput
)

lig = LayerIntegratedGradients(model, model.model.layers[-1])

llm_attr = LLMGradientAttribution(lig, model.tokenizer)


In [ ]:
prompt = "1+1="
target = "2"
skip_tokens = [1]

input = TextTokenInput(
    "1+1=",
    model.tokenizer,
    skip_tokens=skip_tokens
)
attr_res = llm_attr.attribute(inp, target=target, skip_tokens=skip_tokens)

attr_res.plot_seq_attr(show=True)